## Import Related Packages

In [ ]:
from keras.optimizers import SGD, Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from cifar10 import *
from modified_resnet import *
from utils import *

## Train Stage 01

In [ ]:
(x_train, y_train), (x_val, y_val) = parse_dataset()

nb_classes = 10
input_shape = (32, 32, 3)
depth = 76
nb_blocks = 3
nb_filter = 128

model = modified_resnet(nb_classes=nb_classes, img_dim=input_shape, depth=depth, nb_blocks=nb_blocks,
                        nb_filter=nb_filter)
batch_size = 64
epochs = 240
sgd = SGD(lr=lr_stepwise(0), momentum=0.9, nesterov=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=batch_size,
                          write_graph=True, write_grads=False, write_images=True,
                          embeddings_freq=0, embeddings_layer_names=None,
                          embeddings_metadata=None, embeddings_data=None,
                          update_freq='epoch')

lr_scheduler = LearningRateScheduler(lr_stepwise)
callbacks = [lr_scheduler, tensorboard]
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

data_augmentation = True

if not data_augmentation:
    model.fit(x_train, y_train,
              batch_size, epochs,
              validation_data=[x_val, y_val],
              callbacks=callbacks)
else:
    print("Training with data augmentation.")
    datagen = ImageDataGenerator(zca_epsilon=1e-06,
                                 # randomly shift images horizontally
                                 width_shift_range=0.1,
                                 # randomly shift images vertically
                                 height_shift_range=0.1,
                                 # set range for random shear
                                 shear_range=0.,
                                 # set range for random zoom
                                 zoom_range=0.2,
                                 # set range for random channel shifts
                                 channel_shift_range=0.,
                                 # set mode for filling points outside the input boundaries
                                 fill_mode='nearest',
                                 # value used for fill_mode = "constant"
                                 cval=0.,
                                 # randomly flip images
                                 horizontal_flip=True)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_val, y_val), steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

## Train Stage 02

In [ ]:
epochs = 150
batch_size = 64

sgd = SGD(lr=lr_cosine_schedule_vary(0), momentum=0.9, nesterov=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=batch_size,
                            write_graph=True, write_grads=False, write_images=True,
                            embeddings_freq=0, embeddings_layer_names=None,
                            embeddings_metadata=None, embeddings_data=None,
                            update_freq='epoch')

    
lr_scheduler = LearningRateScheduler(lr_cosine_schedule_vary)
callbacks = [lr_scheduler, tensorboard]
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
data_augmentation = True

if not data_augmentation:
    model.fit(x_train, y_train,
              batch_size, epochs,
              validation_data=[x_val, y_val],
              callbacks=callbacks)
else:
    print("Training with data augmentation.")
    datagen = ImageDataGenerator(zca_epsilon=1e-06,
                                 # randomly shift images horizontally
                                 width_shift_range=0.1,
                                 # randomly shift images vertically
                                 height_shift_range=0.1,
                                 # set range for random shear
                                 shear_range=0.,
                                 # set range for random zoom
                                 zoom_range=0.2,
                                 # set range for random channel shifts
                                 channel_shift_range=0.,
                                 # set mode for filling points outside the input boundaries
                                 fill_mode='nearest',
                                 # value used for fill_mode = "constant"
                                 cval=0.,
                                 # randomly flip images
                                 horizontal_flip=True)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_val, y_val), steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs, verbose=1, workers=6,
                        callbacks=callbacks)